**Import necessary packages/modules**

In [ ]:
# Cell 1
import pickle
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from google.colab import drive
from matplotlib.collections import LineCollection, PatchCollection
from matplotlib.patches import Rectangle

**Connect this notebook to your Google Drive**

In [ ]:
# Cell 2
drive.mount("/content/gdrive", force_remount=True)
notebook_path = Path("/content/gdrive/MyDrive/SciComp101-GC")
notebook_path = notebook_path / Path("Session 17 - Maze Searching")
notebook_path

**Define a function to link two cells to each other in the adjacency matrix**

In [ ]:
# Cell 3
def link_cells(adj, y1, x1, y2, x2):
    cell1 = y1 * 10 + x1
    cell2 = y2 * 10 + x2
    adj[cell1, cell2] = True
    adj[cell2, cell1] = True

**Define a function to initialize an adjacency matrix given a maze stored as a numpy `array`**

In [ ]:
# Cell 4
def init_adjmat(maze):
    adj = np.zeros(10000, dtype=np.bool_).reshape((100, 100))
    for y in range(10):
        for x in range(10):
            cell = int(maze[y, x])
            if y > 0 and cell & 1 != 1:
                link_cells(adj, y, x, y - 1, x)
            if y < 9 and cell & 4 != 4:
                link_cells(adj, y, x, y + 1, x)
            if x < 9 and cell & 2 != 2:
                link_cells(adj, y, x, y, x + 1)
            if x > 0 and cell & 8 != 8:
                link_cells(adj, y, x, y, x - 1)

    return adj

**Define a function to calculate the maximum necessary steps to reach the exit**
1. This count will be the number of steps in the shortest path to the exit
2. We determine this count by multiplying the adjacency matrix by itself repeatedly
3. We keep multiplying until a `True` value appears in the adj matrix at **both** the entrance and exit elements

In [ ]:
# Cell 5
def max_steps(adj_matrix):
    adj = np.copy(adj_matrix)
    steps = 0
    # Keep multiplying until the entrance and exit are "linked"
    while not (adj[0, 99] & adj[99, 0]):
        # Use @ operator for matrix multiplication
        adj = adj @ adj_matrix
        steps += 1
    # Add +2 to max steps, because we need +1 to step into
    # the entrance cell, and another +1 to enter the exit cell
    steps += 2
    return steps

**Define a function to plot the walls of a given cell in the maze**

In [ ]:
# Cell 6
def plot_cell_walls(ax, maze):
    for y in range(10):
        bottom = (9 - y) * 10
        top = bottom + 10
        for x in range(10):
            left = x * 10
            right = left + 10
            cell = int(maze[y, x])
            if cell & 1 == 1:
                lc = LineCollection(
                    [[(left, top), (right, top)]],
                    color="black",
                    linewidth=3,
                )
                ax.add_collection(lc)
            if cell & 2 == 2:
                lc = LineCollection(
                    [[(right, bottom), (right, top)]],
                    color="black",
                    linewidth=3,
                )
                ax.add_collection(lc)
            if cell & 4 == 4:
                lc = LineCollection(
                    [[(left, bottom), (right, bottom)]],
                    color="black",
                    linewidth=3,
                )
                ax.add_collection(lc)
            if cell & 8 == 8:
                lc = LineCollection(
                    [[(left, bottom), (left, top)]],
                    color="black",
                    linewidth=3,
                )
                ax.add_collection(lc)

**Define a function to plot the steps taken in the first found path**
1. We don't record the coordinates of steps taken in error that resulted in <u>backtracking</u>
2. However, we do keep a count of every step taken, even if in error

In [ ]:
# Cell 7
def plot_steps(ax, steps):
    for step in steps:
        y, x, _ = step
        bottom = (9 - y) * 10
        left = x * 10
        patch = Rectangle((left + 4, bottom + 4), 2, 2)
        ax.add_collection(PatchCollection([patch], facecolor="blue"))

**Define a function to plot every cell in the maze**

In [ ]:
# Cell 8
def plot_maze(ax, maze, steps):
    ax.axis("off")
    ax.set_aspect("equal")
    ax.set_xlim(-5, 105)
    ax.set_ylim(-5, 105)
    ax.set_title(f"{len(steps)} steps ({total_steps} total)")

    # Plot enter and exit cells
    entrance = Rectangle((0, 90), 10, 10)
    ax.add_collection(PatchCollection([entrance], facecolor="tan"))
    exit = Rectangle((90, 0), 10, 10)
    ax.add_collection(PatchCollection([exit], facecolor="orange"))

    # Plot cell corner circles
    for x in range(0, 110, 10):
        for y in range(0, 110, 10):
            ax.scatter(x, y, color="black")

    plot_cell_walls(ax, maze)
    plot_steps(ax, steps)

**Define a function to determine if we have previously visited a given cell**

In [ ]:
# Cell 9
def in_path(steps, y, x):
    for step in reversed(steps):
        sy, sx, _ = step
        if sy == y and sx == x:
            return True
    return False

**Define a function to do a <u>depth-first</u> search of the maze**
1. The record of prior cells visited (along with the last tried direction) is in `steps` list
2. The steps list prevents us from getting in endless loop trying the same path over and over again
3. We can use the `step_limit` count to realize when we are proceeding down a bad path and need to backtrack
4. The function returns True when we have entered the exit cell, otherwise it returns False

In [ ]:
# Cell 10
def search_maze(maze, steps, step_limit):
    global total_steps

    y, x, dir = steps.pop()
    total_steps += 1

    if x == 9 and y == 9:
        steps.append((9, 9, 0))
        return True

    # Now we can use the adjacency matrix to know when
    # to start backtracking since the current path is too long!
    if len(steps) == step_limit:
        return False

    if dir == 0:
        steps.append((y, x, 1))
        if int(maze[y, x]) & 1 != 1 and not in_path(steps, y - 1, x):
            steps.append((y - 1, x, 0))
        return False

    if dir == 1:
        steps.append((y, x, 2))
        if int(maze[y, x]) & 2 != 2 and not in_path(steps, y, x + 1):
            steps.append((y, x + 1, 0))
        return False

    if dir == 2:
        steps.append((y, x, 4))
        if int(maze[y, x]) & 4 != 4 and not in_path(steps, y + 1, x):
            steps.append((y + 1, x, 0))
        return False

    if dir == 4:
        steps.append((y, x, 8))
        if int(maze[y, x]) & 8 != 8 and not in_path(steps, y, x - 1):
            steps.append((y, x - 1, 0))
        return False

    return False

**Navigate the maze (find the first path that connects the entrance to exit)**
1. Generate a `numpy array` from a <u>pickle</u> file (a compressed binary format)
2. Initialize the `steps` (s) list of tuples
3. Each tuple contains (for each step) the $[row,col]$ cell coordinates and the direction taken at that cell
4. Initialize the adjacency matrix and determine the maximum number of steps on the shortest path
5. We increment the global variable `total_steps` every time we enter a cell, even if we have already visited that cell
6. We stay in the while() loop until search_maze() returns True which happens when we reach the exit
7. While search_maze() returns False, we keep trying the next step


In [ ]:
# Cell 11
file_name = "maze.pickle"
file_path = notebook_path / file_name
with Path.open(file_path, "rb") as infile:
    m = pickle.load(infile)

# Steps is list of tuples, where each tuple contains the
# (y, x, last direction tried) for each step along current path
s = [(0, 0, 0)]

# Create boolean adjacency matrix from the wall data
adj_mat = init_adjmat(m)

# Calculate maximum number of allowed steps along shortest path
limit = max_steps(adj_mat)

total_steps = 0

while not search_maze(m, s, limit):
    pass

plt.figure(figsize=(8, 8))
plot_maze(plt.gca(), m, s)
plt.show()